In [1]:
import numpy as np
import pandas as pd
import taxcalc as tc
import microdf as mdf

In [2]:
run 'convert_asec_taxcalc.py'

In [3]:
run 'make_tax_units.py'

In [24]:
person = pd.read_csv('~/UBICenter/covid_ui/asec_2019_ipums.csv.gz')

## Preprocessing

In [25]:
# Set columns to lowercase and to 0 or null as appropriate.
prep_ipum(person)
# Add taxid and related fields.
tax_unit_id(person)
# Add other person-level columns in taxcalc form.
person = convert_asec_person_taxcalc(person)
# 99 is the missing code for wksunem1.
# Note: Missing codes for features used in taxcalc are recoded in
# convert_asec_taxcalc.py.
person.loc[person.wksunem1 == 99, 'wksunem1'] = 0

## Add UI to person records

In [26]:
FPUC_MAX_WEEKS = 17  # April to July.
FPUC2_MAX_WEEKS = 17 + 22  # Adding August to December.
FPUC_WEEKLY_BEN = 600
person['fpuc'] = FPUC_WEEKLY_BEN * np.minimum(person.wksunem1, FPUC_MAX_WEEKS)
person['fpuc2'] = FPUC_WEEKLY_BEN * np.minimum(person.wksunem1,
                                               FPUC2_MAX_WEEKS)

In [27]:
# Store original unemployment benefits.
person['e02300_orig'] = person.e02300

## Create tax units and calculate tax liability

In [28]:
# Create tax unit dataframe.
tu = create_tax_unit(person)
tu['RECID'] = tu.taxid
tu['tax'] = mdf.calc_df(records=tc.Records(tu), year=2018).tax.values

Simulate FPUC.

In [29]:
# Create tax unit dataframe.
person.e02300 = person.e02300_orig + person.fpuc
tu_fpuc = create_tax_unit(person)
tu['e02300_fpuc'] = tu_fpuc.e02300
tu_fpuc['RECID'] = tu_fpuc.taxid
tu['tax_fpuc'] = mdf.calc_df(records=tc.Records(tu_fpuc),
                             year=2018).tax.values
del tu_fpuc

Simulate extended FPUC.

In [30]:
# Create tax unit dataframe.
person.e02300 = person.e02300_orig + person.fpuc2
tu_fpuc2 = create_tax_unit(person)
tu['e02300_fpuc2'] = tu_fpuc2.e02300
tu_fpuc2['RECID'] = tu_fpuc2.taxid
tu['tax_fpuc2'] = mdf.calc_df(records=tc.Records(tu_fpuc2),
                             year=2018).tax.values
del tu_fpuc2

In [41]:
# Change person e02300 back.
person.e02300 = person.e02300_orig

## Merge back to the person level

Have each person pay the share of tax differences in proportion with their FPUC.

In [42]:
tu['fpuc_total'] = tu.e02300_fpuc - tu.e02300
tu['fpuc2_total'] = tu.e02300_fpuc2 - tu.e02300
tu['fpuc_tax_total'] = tu.tax_fpuc - tu.tax
tu['fpuc2_tax_total'] = tu.tax_fpuc2 - tu.tax
tu.set_index('RECID', inplace=True)

In [91]:
person2 = person.merge(tu[['fpuc_total', 'fpuc2_total',
                           'fpuc_tax_total', 'fpuc2_tax_total']],
                       left_on='taxid', right_index=True)

In [92]:
for i in ['fpuc', 'fpuc2']:
    person2[i + '_tax'] = np.where(person2[i + '_total'] == 0, 0,
        person2[i + '_tax_total'] * person2[i] / person2[i + '_total'])
    person2[i + '_net'] = person2[i] - person2[i + '_tax']

## Calculate budget-neutral UBIs and payroll taxes

In [93]:
fpuc_budget = mdf.weighted_sum(person2, 'fpuc_net', 'asecwt')
fpuc2_budget = mdf.weighted_sum(person2, 'fpuc2_net', 'asecwt')
pop = person2.asecwt.sum()
adult_pop = person2[person2.age > 17].asecwt.sum()
total_fica = mdf.weighted_sum(person2, 'fica', 'asecwt')

In [94]:
fpuc_ubi = fpuc_budget / pop
fpuc_adult_ubi = fpuc_budget / adult_pop
fpuc_fica_pct_cut = fpuc_budget / total_fica

fpuc2_ubi = fpuc2_budget / pop
fpuc2_adult_ubi = fpuc2_budget / adult_pop
fpuc2_fica_pct_cut = fpuc2_budget / total_fica

In [95]:
person2['fpuc_ubi'] = fpuc_ubi
person2['fpuc_adult_ubi'] = np.where(person2.age > 17, fpuc_adult_ubi, 0)
person2['fpuc_fica_cut'] = person2.fica * fpuc_fica_pct_cut

person2['fpuc2_ubi'] = fpuc2_ubi
person2['fpuc2_adult_ubi'] = np.where(person2.age > 17, fpuc2_adult_ubi, 0)
person2['fpuc2_fica_cut'] = person2.fica * fpuc2_fica_pct_cut

In [99]:
person2[['fpuc_net', 'fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_cut']].sample(10)

,fpuc_net,fpuc_ubi,fpuc_adult_ubi,fpuc_fica_cut
179509,0.000000,219.917603,284.685068,1074.596356
149590,0.000000,219.917603,0.000000,0.000000
16628,0.000000,219.917603,284.685068,1171.571072
150185,4550.678243,219.917603,284.685068,247.560594
66676,0.000000,219.917603,284.685068,0.000000
17040,0.000000,219.917603,0.000000,0.000000
133003,0.000000,219.917603,284.685068,0.000000
64858,0.000000,219.917603,284.685068,511.625227
146342,9000.000000,219.917603,284.685068,8.305955
165397,0.000000,219.917603,284.685068,288.874628


## Aggregate to SPM units

In [72]:
mdf.weighted_sum(person2, 'fpuc2_net', 'asecwt') / 1e9

103.159433661786

In [ ]:
mdf.weighted_sum(person2, 'fpuc2_net', 'asecwt') / 1e9

In [50]:
person2[(person2.fpuc < person2.fpuc_total) & (person2.fpuc > 0)]

,year,serial,month,cpsid,asecflag,asecwth,statefip,pernum,cpsidp,asecwt,...,e02300_total,e02300_fpuc_total,e02300_fpuc2_total,tax_total,tax_fpuc_total,tax_fpuc2_total,fpuc_total,fpuc2_total,fpuc_tax_total,fpuc2_tax_total
729,2019,567,3,0,1,588.46,23,3,0,1484.07,...,0,2400,2400,27702.634143,27827.295022,27827.295022,2400,2400,124.660879,124.660879
730,2019,567,3,0,1,588.46,23,4,0,1057.91,...,0,2400,2400,27702.634143,27827.295022,27827.295022,2400,2400,124.660879,124.660879
1968,2019,1459,3,20171204023900,1,424.03,33,2,20171204023903,545.73,...,0,1800,1800,2317.066667,2435.976146,2435.976146,1800,1800,118.909479,118.909479
1969,2019,1459,3,20171204023900,1,424.03,33,3,20171204023902,587.95,...,0,1800,1800,2317.066667,2435.976146,2435.976146,1800,1800,118.909479,118.909479
4179,2019,2705,3,20190306373800,1,237.69,50,1,20190306373801,237.69,...,0,10800,15600,39256.110693,39817.084647,40066.406404,10800,15600,560.973954,810.295712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177848,2019,93566,3,20190101962900,1,1087.15,15,2,20190101962902,1087.15,...,0,14400,14400,25635.885333,26383.850605,26383.850605,14400,14400,747.965272,747.965272
177876,2019,93577,3,20180301990100,1,399.66,15,2,20180301990102,399.66,...,4410,10410,10410,29851.945177,30163.597374,30163.597374,6000,6000,311.652197,311.652197
177878,2019,93577,3,20180301990100,1,399.66,15,4,20180301990104,585.61,...,4410,10410,10410,29851.945177,30163.597374,30163.597374,6000,6000,311.652197,311.652197
178095,2019,93701,3,20180201936400,1,548.87,15,3,20180201936408,469.31,...,0,15000,20400,7705.898781,8130.879050,8283.871946,15000,20400,424.980268,577.973165


In [19]:
tu.e02300_fpuc.sum() / tu.e02300.sum()

5.658077231808007

In [31]:
tu.tax_fpuc.sum() / tu.tax.sum()

1.0006684382444695

In [32]:
tu.tax_fpuc2.sum() / tu.tax.sum()

1.0009234493486514

In [14]:
tu.e02300_fpuc.describe()

count    86399.000000
mean       537.477193
std       2397.317471
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      54088.000000
Name: e02300_fpuc, dtype: float64